## 1 - Packages

These are some of the most important packages that you're going to need in order to use ***```pydANN```***

- [numpy](www.numpy.org) (or numeric python) is the main package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a library to plot graphs -- which are optional -- in Python.

- np.random.seed(x), where 'x' is a constant integer, is used to keep all the random function calls consistent as we're going to have some random events occuring.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class ann:
    
    def __init__(self):
        self.hidden_Layers = []
        self.total_layers = []
        self.learning_rate = 0.001
        self.W = {}
        self.b = {}
        self.Z = {}
        self.A = {}
        
    def addHL(self,hl):
        self.hidden_Layers = hl
        
    
    def init_Params(self,x):
        
        # creating the weights and biases with seed(1)
        np.seed(144)
        for i in range(1,len(self.hidden_Layers)+2):
            self.W[str(i)] = np.random.randn()
    
    
    def forePropagate(self,train_x,train_y):
        pass